In [ ]:
import numpy as np
from PIL import Image
import torch,torchvision
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
#https://www.kaggle.com/datasets/steph039/primate-species

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_dataset = datasets.ImageFolder(root=r"/kaggle/input/primate-species/dataset/train/", transform=transform)
test_dataset = datasets.ImageFolder(root=r"/kaggle/input/primate-species/dataset/test/", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers = 2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True,num_workers = 2)

In [ ]:
classes = ['Bald Uakari','Gorilla', 'Mandrill', 'Golden Lion Tamarin', 'Emperor Tamarin', 'Silvery Marmoset', 'Proboscis Monkey']

In [ ]:
def show_image(img, label, classes):
    img = img.permute(1, 2, 0).cpu().numpy()
    plt.imshow(img)
    plt.title(f"Label: {classes[label]} ({label})")
    plt.show()

image, label = train_dataset[300]
classes = train_dataset.classes    
show_image(image, label, classes)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84, 7) #output: 7 classes

    def forward(self, x):
        x = self.pool(f.relu(self.conv1(x)))
        x = self.pool(f.relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = f.relu(self.fc1(x))  
        x = f.relu(self.fc2(x))  
        x = self.fc3(x) 
        return x

In [ ]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.001)

In [ ]:
def train(net, train_loader, test_loader, loss_function, optimizer, epochs):
    for epoch in range(epochs):
        print(f'\nTraining epoch {epoch + 1}/{epochs}')
        running_loss = 0.0
        net.train()  

        # Training loop
        for i, data in enumerate(train_loader):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)  
            loss = loss_function(outputs, labels)  
            loss.backward()
            optimizer.step()  
            running_loss += loss.item()  
        avg_loss = running_loss / len(train_loader)
        print(f'Loss: {avg_loss:.4f}')  
        evaluate(net, test_loader, loss_function)
        
def evaluate(net, test_loader, loss_function):
    correct = 0
    total = 0
    total_loss = 0.0
    net.eval() 

    with torch.no_grad():  
        for data in test_loader:
            images, labels = data
            outputs = net(images) 
            loss = loss_function(outputs, labels) 
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)  # Get predicted class
            correct += (predicted == labels).sum().item()  # Count correct predictions
            total += labels.size(0)

    accuracy = (correct / total) * 100
    avg_loss = total_loss / len(test_loader)
    print(f'Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

train(net, train_loader, test_loader, loss_function, optimizer,15)

In [ ]:
torch.save(net.state_dict(), 'model.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('model.pth'))

In [ ]:
net.eval()

transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

img_path = r''
image = Image.open(img_path)
image_tensor = transform(image).unsqueeze(0)

with torch.no_grad():
    output = net(image_tensor)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    _, predicted_class = torch.max(probabilities, 1)
    classes = ['Bald Uakari', 'Gorilla', 'Mandrill', 'Golden Lion Tamarin', 'Emperor Tamarin', 'Silvery Marmoset', 'Proboscis Monkey']
    classes.sort()
    predicted_label = classes[predicted_class.item()]
    print(f"Predicted Label: {predicted_label}")
    plt.imshow(image)
    plt.show()

In [ ]:
val_dataset = datasets.ImageFolder(root=r"/kaggle/input/primate-species/dataset/val/", transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)  

predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = net(inputs) 
        _, predicted = torch.max(outputs, 1) 
        predictions.extend(predicted.numpy()) 
        true_labels.extend(labels.numpy())  
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Compute metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted') 
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Compute confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Plot confusion matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()